# Gaertner L116A Ellipsometer

**Scott Prahl**

**May 2020**

**Version 3**

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pypolar.ellipsometry as ell
import pypolar.gaerter as gaerter

## Hardware

This only of interest to the Oregon Tech students.

Our Gaertner L116A ellipsometer was made in 1978.  Since the computer that was attached to this ellipsometer is long gone, we programmed an Arduino to serve as its computer interface.  The Arduino collects 72 data points from one rotation of the ellipsometer analyzer.  This array is sent over the USB serial connection to a host computer running a Jupyter notebook.

<img src="https://omlc.org/~prahl/host/453/week3/ellipsometer.jpg" width="50%">

## Optical Elements

<img src="https://omlc.org/~prahl/host/453/week3/image-A-0014.png" width="50%">

### Establishing a connection to the ellipsometer

You must establish serial communication between your computer and the Arduino.
**To do this you need to know the name of the serial connection.**

One way to figure our the right name is to list the current serial ports on your computer:  `gaerter.current_serial_ports()` returns such a list.

Of course, if the ellipsometer is not plugged into the computer then the proper serial port will not appear.

In [2]:
s = gaerter.current_serial_ports()
print(s)

                           portname  info
  /dev/cu.RasputinsAirpods-Wirele-5  n/a
    /dev/cu.AnnesBeatsSolo3-SPPServ  n/a
  /dev/cu.RaspustinsOldAirPods-Wi-1  n/a
    /dev/cu.Bluetooth-Incoming-Port  n/a
    /dev/cu.AnnesBeatsSolo3-Wireles  n/a
    /dev/cu.RasputinsAirpods-Wirele  n/a
  /dev/cu.RasputinsAirpods-Wirele-1  n/a
  /dev/cu.RasputinsAirpods-Wirele-2  n/a
  /dev/cu.RasputinsAirpods-Wirele-3  n/a
  /dev/cu.RasputinsAirpods-Wirele-4  n/a
  /dev/cu.RasputinsAirpods-Wirele-6  n/a
  /dev/cu.RasputinsAirpods-Wirele-7  n/a
  /dev/cu.RasputinsAirpods-Wirele-8  n/a



In [3]:
### Identify the port that is connected to the Arduino
### Copy that portname and port connected to the Ardunio string from above as the correct serial port id below

id = '/dev/tty.usbmodem14101'   # UPDATE ME

conn = gaerter.connect_to_ellipsometer(id)

ERROR: serial port '/dev/tty.usbmodem14101' is disconnected


### Summary

This notebook won't collect data unless everything is turned on and you establish basic serial communication between your computer and the ellipsometer.

* install `pyserial`

* connect your computer to the ellipsometer

* identify the name of the serial port

* turn on ellipsometer and laser

* start analyzer motor

* collect data


### Convenience Analysis Routines

In [4]:
# the angle phi is uniformly distributed from 0 to 360 across 72 datapoints
degrees = np.linspace(0,360,num=72,endpoint=False)
phi = np.radians(degrees)

signal_1 = np.array([2297,2185,2037,1903,1769,1626,1475,1252,1011,787,596,431,292,174,88,32,20,41,102,199,327,491,667,878,1077,1296,1512,1698,1911,2067,2223,2342,2382,2420,2445,2419,2324,2266,2118,2011,1785,1600,1411,1178,989,777,600,435,286,168,83,31,21,40,101,191,313,452,643,831,1041,1241,1393,1593,1736,1920,2048,2180,2291,2296,2369,2260])
signal_2 = np.array([2449,2520,2515,2484,2431,2342,2189,2042,1859,1655,1447,1229,1018,807,628,450,296,175,85,34,19,42,105,197,322,470,662,869,1087,1290,1462,1625,1803,1978,2129,2271,2371,2415,2420,2387,2288,2176,2048,1927,1787,1654,1475,1254,1025,792,597,436,297,174,89,34,19,41,103,200,332,493,677,874,1087,1304,1517,1708,1915,2087,2219,2352])

### Obtaining a signal

To acquire 72 equally spaced measurements as the analyzer is rotated, just do
```
signal=get_reflectance(conn)
```

In [5]:
# Testing
P = 350
theta_i = 90
QWP = False
offset=30
#signal=get_reflectance(conn)
#filename=save_data_with_name(signal,'silicon',theta_i,P,QWP)

filename = 'silicon, I=30.0°, P=0.0°, QWP=True, 2019-04-23-15-19-42.txt'
signal = gaerter.read_data_with_name(filename)

print(signal)
I_0, alpha, beta = ell.find_fourier(phi,signal)
fitted_signal = I_0*(1+alpha*np.cos(2*phi)+beta*np.sin(2*phi))

plt.scatter(degrees,signal,s=5, color='black')
plt.plot(degrees,fitted_signal, color='red')

plt.xlabel("Analyzer Angle (degrees)")
plt.ylabel("Ellipsometer Measurement")
plt.title(r"Silicon, $\theta_i$=%.0f, P=%.1f°, QWP=%r" % (theta_i, P, QWP) )
plt.axvline(P,color='blue')
plt.axvline((P+offset)%360,linestyle=':',color='blue')

plt.xlim(-10,370)
plt.show()

print("   I_0 = %6.2f" % I_0)
print(" alpha = %6.2f" % alpha)
print("  beta = %6.2f" % beta)

ERROR: file 'silicon, I=30.0°, P=0.0°, QWP=True, 2019-04-23-15-19-42.txt' does not exist
None


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'